In [2]:
from mesa.batchrunner import BatchRunner
from model import MinimalModel

parameters = {
    #"magnitude": [5, 6, 7],
    "num_citizens": [3000],
    "num_buildings": [100],
    "num_hospitals": [5],
    "num_ambulances": [0,1,15],
    "num_doctors": [0],
    "dispatch_size": [1,10,75]
}

'Define the parameter settings to be tested'

model_reporters = {"num_citizens": lambda m: m.schedule.get_agent_count(),
                   # "num_buildings": lambda m: m.schedule.get_agent_count(),
                   # "num_hospitals": lambda m: m.schedule.get_agent_count(),
                   # "num_ambulances": lambda m: m.schedule.get_agent_count(),
                   # "num_doctors": lambda m: m.schedule.get_agent_count(),
                   # "dispatcher_size": lambda m: m.dispatcher.size,
                   # "num_dead_citizens": lambda m: m.dead_citizens,
                   # "num_saved_citizens": lambda m: m.saved_citizens,
                   # "num_injured_citizens": lambda m: m.injured_citizens,
                   # "num_dead_doctors": lambda m: m.dead_doctors,
                   # "num_saved_doctors": lambda m: m.saved_doctors,
                   # "num_injured_doctors": lambda m: m.injured_doctors,
                   # "num_destroyed_buildings": lambda m: m.destroyed_buildings,
                   # "num_damaged_buildings": lambda m: m.damaged_buildings
                   }

'keep track of the agents position and the state of the city'

class BatchrunnerExtender(BatchRunner):
        # def _result_prep_mp(self, results):
        #     """
        #     Helper Function
        #     :param results: Takes results dictionary from Processpool and single processor debug run and fixes format to
        #  make compatible with BatchRunner Output
        #  :updates model_vars and agents_vars so consistent across all batchrunner
        #  """
        #     # Take results and convert to dictionary so dataframe can be called
        #     for model_key, model in results.items():
        #         if self.model_reporters:
        #             self.model_vars[model_key] = self.collect_model_vars(model)
        #         if self.agent_reporters:
        #             agent_vars = self.collect_agent_vars(model)
        #             for agent_id, reports in agent_vars.items():
        #                 agent_key = model_key + (agent_id,)
        #                 self.agent_vars[agent_key] = reports
        #         if hasattr(model, "datacollector"):
        #             if model.datacollector.model_reporters is not None:
        #                 self.datacollector_model_reporters[
        #                     model_key
        #                 ] = model.datacollector.get_model_vars_dataframe()
        #             if model.datacollector.agent_reporters is not None:
        #                 self.datacollector_agent_reporters[
        #                     model_key
        #                 ] = model.datacollector.get_agent_vars_dataframe('Citizen')

        # # Make results consistent
        #     if len(self.datacollector_model_reporters.keys()) == 0:
        #         self.datacollector_model_reporters = None
        #     if len(self.datacollector_agent_reporters.keys()) == 0:
        #         self.datacollector_agent_reporters = None

    def run_iteration(self, kwargs, param_values, run_count):
        """ Run one iteration of the model, with the given parameters. """
        model = self.model_cls(**kwargs)
        results = self.run_model(model)
        if param_values is not None:
            model_key = tuple(param_values) + (run_count,)
        else:
            model_key = (run_count,)
    
        if self.model_reporters:
            self.model_vars[model_key] = self.collect_model_vars(model)
        if self.agent_reporters:
            agent_vars = self.collect_agent_vars(model)
            for agent_id, reports in agent_vars.items():
                agent_key = model_key + (agent_id,)
                self.agent_vars[agent_key] = reports
        # Collects data from datacollector object in model
        if results is not None:
            if results.model_reporters is not None:
                self.datacollector_model_reporters[
                    model_key
                ] = results.get_model_vars_dataframe()
            if results.agent_reporters is not None:
                self.datacollector_agent_reporters[
                    model_key
                ] = results.get_agent_vars_dataframe('Citizen')

batch_run = BatchrunnerExtender(MinimalModel, parameters, max_steps=500, model_reporters= model_reporters)

'specifying the model class, the parameter settings, the number of times to run each set of parameters, and the reporters to be tracked'

batch_run.run_all()

'run the Batchrunner'


'Collect and analyze the results of the batch run to get the dataframe of the results'



C:\Users\Kaninik\AppData\Local\Temp\ipykernel_27912\687171613.py:94: DeprecationWarning: BatchRunner class has been replaced by batch_run function. Please see documentation.
  batch_run = BatchrunnerExtender(MinimalModel, parameters, max_steps=500, model_reporters= model_reporters)
0it [00:00, ?it/s]

This is step: 0
This is step: 1
This is step: 2
This is step: 3
This is step: 4
This is step: 5
This is step: 6
This is step: 7
This is step: 8
This is step: 9
This is step: 10
This is step: 11
This is step: 12
This is step: 13
This is step: 14
This is step: 15
This is step: 16
This is step: 17
This is step: 18
This is step: 19
This is step: 20
This is step: 21
This is step: 22
This is step: 23
This is step: 24
This is step: 25
This is step: 26
This is step: 27
This is step: 28
This is step: 29
This is step: 30
This is step: 31
This is step: 32
This is step: 33
This is step: 34
This is step: 35
This is step: 36
This is step: 37
This is step: 38
This is step: 39
This is step: 40
This is step: 41
This is step: 42
This is step: 43
This is step: 44
This is step: 45
This is step: 46
This is step: 47
This is step: 48
This is step: 49
This is step: 50
This is step: 51
This is step: 52
This is step: 53
This is step: 54
This is step: 55
This is step: 56
This is step: 57
This is step: 58
This is

1it [00:08,  8.75s/it]

This is step: 0
This is step: 1
This is step: 2
This is step: 3
This is step: 4
This is step: 5
This is step: 6
This is step: 7
This is step: 8
This is step: 9
This is step: 10
This is step: 11
This is step: 12
This is step: 13
This is step: 14
This is step: 15
This is step: 16
This is step: 17
This is step: 18
This is step: 19
This is step: 20
This is step: 21
This is step: 22
This is step: 23
This is step: 24
This is step: 25
This is step: 26
This is step: 27
This is step: 28
This is step: 29
This is step: 30
This is step: 31
This is step: 32
This is step: 33
This is step: 34
This is step: 35
This is step: 36
This is step: 37
This is step: 38
This is step: 39
This is step: 40
This is step: 41
This is step: 42
This is step: 43
This is step: 44
This is step: 45
This is step: 46
This is step: 47
This is step: 48
This is step: 49
This is step: 50
This is step: 51
This is step: 52
This is step: 53
This is step: 54
This is step: 55
This is step: 56
This is step: 57
This is step: 58
This is

2it [00:16,  8.03s/it]

...

This is step: 0
This is step: 1
This is step: 2
This is step: 3
This is step: 4
This is step: 5
This is step: 6
This is step: 7
This is step: 8
This is step: 9
This is step: 10
This is step: 11
This is step: 12
This is step: 13
This is step: 14
This is step: 15
This is step: 16
This is step: 17
This is step: 18
This is step: 19
This is step: 20
This is step: 21
This is step: 22
This is step: 23
This is step: 24
This is step: 25
This is step: 26
This is step: 27
This is step: 28
This is step: 29
This is step: 30
This is step: 31
This is step: 32
This is step: 33
This is step: 34
This is step: 35
This is step: 36
This is step: 37
This is step: 38
This is step: 39
This is step: 40
This is step: 41
This is step: 42
This is step: 43
This is step: 44
This is step: 45
This is step: 46
This is step: 47
This is step: 48
This is step: 49
This is step: 50
This is step: 51
This is step: 52
This is step: 53
This is step: 54
This is step: 55
This is step: 56
This is step: 57
This is step: 58
This is

8it [01:16, 11.09s/it]

This is step: 0
This is step: 1
This is step: 2
This is step: 3
This is step: 4
This is step: 5
This is step: 6
This is step: 7
This is step: 8
This is step: 9
This is step: 10
This is step: 11
This is step: 12
This is step: 13
This is step: 14
This is step: 15
This is step: 16
This is step: 17
This is step: 18
This is step: 19
This is step: 20
This is step: 21
This is step: 22
This is step: 23
This is step: 24
This is step: 25
This is step: 26
This is step: 27
This is step: 28
This is step: 29
This is step: 30
This is step: 31
This is step: 32
This is step: 33
This is step: 34
This is step: 35
This is step: 36
This is step: 37
This is step: 38
This is step: 39
This is step: 40
This is step: 41
This is step: 42
This is step: 43
This is step: 44
This is step: 45
This is step: 46
This is step: 47
This is step: 48
This is step: 49
This is step: 50
This is step: 51
This is step: 52
This is step: 53
This is step: 54
This is step: 55
This is step: 56
This is step: 57
This is step: 58
This is

9it [01:31, 10.14s/it]


'Collect and analyze the results of the batch run to get the dataframe of the results'

In [7]:
results_model = batch_run.datacollector_model_reporters
results_agents = batch_run.datacollector_agent_reporters